In [8]:
def harvest_arxiv(id_list=False):
    "Gets metadata for latest submissions from the arXiv API or for the specified IDs."
    
    import urllib
    import feedparser
    
    base_url = 'http://export.arxiv.org/api/query?' # base api query url

    # search parameters
    start_index = 0               # 0 = most recent API result
    max_index = 1000              # upper bound on paper index we will fetch
    results_per_iteration = 400   # passed to arxiv API
    sort_by = 'submittedDate'   # submittedDate, lastUpdatedDate, relevance
    # ISSUE: sortBy does not work!!!
    

    # query used for arxiv API

    # alternative or additional to the search_query
    if id_list:
    	# note: if (except for id_list) no arguments specified, defaults to start=0 and max_results=10
    	query = 'id_list=%s&start=%i&max_results=%i' % (id_list,start_index,results_per_iteration)
    else:
    	# for documentation see https://arxiv.org/help/api/user-manual#detailed_examples
    	search_query = ('%28cat:cond-mat.dis-nn+OR+'
                		   'cat:cond-mat.stat-mech+OR+'
                		   'cat:cond-mat.str-el+OR+'
                		   'cat:physics.comp-ph+OR+'
                		   'cat:quant-ph%29'
                			'+AND+'
                        '%28%28cat:cs.LG+OR+'
                              'cat:stat.ML%29+OR+'
                        '%28abs:machine+AND+abs:learning%29+OR+'
                        '%28abs:neural+AND+abs:network%29%29') 
                        #'%28cat:cs.LG+OR+'
                        #'cat:stat.ML%29')
    
    	query = 'search_query=%s&sortBy=%s&start=%i&max_results=%i' % (search_query, 
                                                               sort_by,
                                                               start_index, 
                                                               results_per_iteration)
    
    response = urllib.request.urlopen(base_url+query).read() # perform a GET request using the base_url and query
    afp = feedparser.parse(response) # parse the response using feedparser
    
    # print out feed information
    print('Feed title:\n"%s"\n' % afp.feed.title)
    print('Feed last updated:\n%s\n' % afp.feed.updated)
    # print opensearch metadata
    print('Total Results for this query: %s' % afp.feed.opensearch_totalresults)
    print('Items Per Page for this query: %s' % afp.feed.opensearch_itemsperpage)
    print('Start Index for this query: %s'   % afp.feed.opensearch_startindex)

    return afp

def list_all(afp):
    "List all findings for review with index, title, publication date and tags."

    index = 0
    for entry in afp.entries:
        print('\n %i'%(index))
        index = index+1
        print(entry.title)
        print('%i-%i-%i'%(entry.published_parsed.tm_mday, entry.published_parsed.tm_mon, entry.published_parsed.tm_year))
        all_cats = [tag['term'] for tag in entry.tags]
        print((' - ').join(all_cats))

def show_info(afp, index, title=True, summary=True, authors=True, pubdate=False, tags=False):
    "Display title, abstract and authors for a given paper from afp."
    entry = afp.entries[index]
    if title:
        print(entry.title+'\n')
    if summary:
        print(entry.summary+'\n')
    if authors:
        print((', ').join(author.name for author in entry.authors))
    if pubdate:
        print('%i-%i-%i'%(entry.published_parsed.tm_mday, entry.published_parsed.tm_mon, entry.published_parsed.tm_year))
    if tags:
        all_cats = [tag['term'] for tag in entry.tags]
        print((' - ').join(all_cats))

def formatted_block(afp, indices):
    "For given afp entry indices, makes formatted entries and concats them in one string."

    formatted_block = '\n'
    for idx in indices:
        entry = afp.entries[idx]
        authors_string = ', '.join(author.name for author in entry.authors)
        formatted_entry = ('- ["%s"]('+'\n'+'%s'+'\n'+'"%s"),'+'\n'+
                     '%s,'+'\n'+'arXiv: [%s](%s),'+'\n'+'%s/%s') % (entry.title, 
                                                                    entry.link, 
                                                                    entry.summary, 
                                                                    authors_string,
                                                                    entry.id.split('abs/')[-1].split('v')[0], 
                                                                    entry.link,
                                                                    entry.published_parsed.tm_mon, 
                                                                    entry.published_parsed.tm_year)
        formatted_block = formatted_block+formatted_entry+'\n\n'
        
    return formatted_block

def add_papers_to_file(new_papers, filename = 'papers-19.md'):
    "Add new publications to lists in file papers-19.md"

    import pandas as pd
    import codecs # important to handle non-standard characters and unicode encoding properly (damn French names!)

    sublists = ['Applying Machine Learning to Physics',
            'Physics-Inspired Ideas Applied to Machine Learning', 
            'Quantum Computation and Quantum Algorithms for Machine Learning']

    d = {'Sublist': sublists, 'New': new_papers}
    df = pd.DataFrame(data=d)

    filepath = '/Users/annagolubeva/Desktop/physicsml.github.io/develop/content/Pages/'
       
    # buffer the file    
    with codecs.open(filepath+filename, 'r', 'utf-8') as fh0:
        buf = fh0.readlines()
    
    # now re-write the file with lines added
    with codecs.open(filepath+filename, 'w', 'utf-8') as fh:
        idx = 0
        for line in buf:
            if line == ('## '+df.Sublist[idx]+'\n'):
                line = line + df.New[idx] + '\n'
                if idx<df.index[-1]:
                    idx = idx+1
            fh.write(line)
    fh.close()

def add_to_csv_list(afp, indices_selected, filename = 'papers_ids_titles_sublists.csv'):
    "Adds new papers to the csv file."
    
    import pandas as pd
    import codecs # important to handle non-standard characters and unicode encoding properly (damn French names!)

    filepath = '/Users/annagolubeva/Desktop/physicsml.github.io/develop/content/Pages/'

    if (filename=='papers_ids_titles_sublists.csv'):
        sublistn = len(indices_selected[0])*[0]+len(indices_selected[1])*[1]+len(indices_selected[2])*[2]
        indices_selected = [index for onelist in indices_selected for index in onelist]
    
    entries = [afp.entries[index] for index in indices_selected]
    idlist = [entry.id.split('abs/')[-1].split('v')[0] for entry in entries]
    titles = [entry.title for entry in entries]

    # get entries from file and add new
    df = pd.read_csv(filepath+filename)
    a = idlist+[elem for elem in df.ID]
    b = titles+[elem for elem in df.Title]
    
    if (filename=='papers_ids_titles_sublists.csv'):
        c = sublistn+[elem for elem in df.Sublist]
        d = {'ID': a, 'Title': b, 'Sublist': c}
    else:
        d = {'ID': a, 'Title': b}
    
    # update the file
    df = pd.DataFrame(data=d)
    df.index.name = 'Index'

    df.to_csv(filepath+filename, encoding='utf-8')

In [9]:
afp = harvest_arxiv()

# list all entries with title, tags and publishing date
list_all(afp)

Feed title:
"ArXiv Query: search_query=(cat:cond-mat.dis-nn OR cat:cond-mat.stat-mech OR cat:cond-mat.str-el OR cat:physics.comp-ph OR cat:quant-ph) AND ((cat:cs.LG OR cat:stat.ML) OR (abs:machine AND abs:learning) OR (abs:neural AND abs:network))&amp;id_list=&amp;start=0&amp;max_results=400"

Feed last updated:
2022-04-15T00:00:00-04:00

Total Results for this query: 5738
Items Per Page for this query: 400
Start Index for this query: 0

 0
Tight Bounds for Quantum State Certification with Incoherent
  Measurements
14-4-2022
quant-ph - cs.CC - cs.IT - cs.LG - math.IT

 1
Programming Quantum Hardware via Levenberg Marquardt Machine Learning
14-4-2022
quant-ph

 2
Are GCMs obsolete?
14-4-2022
physics.ao-ph - nlin.CD - physics.comp-ph

 3
Efficient and practical quantum compiler towards multi-qubit systems
  with deep reinforcement learning
14-4-2022
quant-ph - cs.LG

 4
Learning topological defects formation with neural networks in a quantum
  phase transition
14-4-2022
cond-mat.dis-nn -


 64
Probabilistic Parameter Estimation Using a Gaussian Mixture Density
  Network: Application to X-ray Reflectivity Data Curve Fitting
30-3-2022
cond-mat.dis-nn - cond-mat.mtrl-sci

 65
Nonequilibrium Statistical Mechanics and Optimal Prediction of
  Partially-Observed Complex Systems
30-3-2022
cond-mat.stat-mech - math.DS - physics.data-an

 66
Circuit encapsulation for efficient quantum computing based on
  controlled many-body dynamics
29-3-2022
quant-ph - cond-mat.str-el - cs.LG

 67
Ab initio calculation of real solids via neural network ansatz
29-3-2022
physics.chem-ph - cond-mat.str-el - physics.comp-ph

 68
Deep Reinforcement Learning for Data-Driven Adaptive Scanning in
  Ptychography
29-3-2022
physics.comp-ph - cs.CV - cs.LG

 69
High Speed Photonic Neuromorphic Computing Using Recurrent Optical
  Spectrum Slicing Neural Networks
29-3-2022
cs.ET - physics.comp-ph - physics.optics

 70
Multiclass classification using quantum convolutional neural networks
  with hybrid quantu

  tomography with convex optimization
23-2-2022
quant-ph

 223
Easily generating and absorbing waves using machine learning
23-2-2022
physics.flu-dyn - physics.comp-ph

 224
Bayesian Target-Vector Optimization for Efficient Parameter
  Reconstruction
23-2-2022
physics.comp-ph - physics.data-an - stat.ML

 225
Optimal QRAM and improved unitary synthesis by quantum circuits with any
  number of ancillary qubits
23-2-2022
quant-ph

 226
Understanding the fragile-to-strong transition in silica from
  microscopic dynamics
22-2-2022
cond-mat.dis-nn - cond-mat.mtrl-sci

 227
A tensor network discriminator architecture for classification of
  quantum data on quantum computers
22-2-2022
quant-ph - cond-mat.stat-mech

 228
Quantum Differential Privacy: An Information Theory Perspective
22-2-2022
quant-ph - cs.CR - cs.IT - cs.LG - math.IT

 229
Extraction of the interaction parameters for $α-$RuCl$_3$ from
  neutron data using machine learning
22-2-2022
cond-mat.str-el - cond-mat.dis-nn - physics

quant-ph - cs.ET

 296
Identification of topological phases using classically-optimized
  variational quantum eigensolver
7-2-2022
quant-ph

 297
Estimating the Euclidean Quantum Propagator with Deep Generative
  Modelling of Feynman paths
6-2-2022
quant-ph - cond-mat.dis-nn - cs.LG - physics.data-an

 298
Transitions in Entanglement Complexity in Random Circuits
5-2-2022
quant-ph

 299
On the relationship between the ground-state wave function of a magnet
  and its static structure factor
4-2-2022
cond-mat.str-el - quant-ph

 300
Direct observation of a dynamical glass transition in a nanomagnetic
  artificial Hopfield network
4-2-2022
cond-mat.dis-nn - cond-mat.mes-hall - cond-mat.stat-mech - cs.NE - nlin.CD

 301
Compact quantum distance-based binary classifier
4-2-2022
quant-ph

 302
Multi-task graph neural networks for simultaneous prediction of global
  and atomic properties in ferromagnetic systems
4-2-2022
cond-mat.mtrl-sci - cs.LG - physics.comp-ph

 303
PSO-PINN: Physics-Info

In [11]:
show_info(afp,1)

Programming Quantum Hardware via Levenberg Marquardt Machine Learning

Significant challenges remain with the development of macroscopic quantum
computing, hardware problems of noise, decoherence, and scaling, software
problems of error correction, and, most important, algorithm construction.
Finding truly quantum algorithms is quite difficult, and many quantum
algorithms, like Shor prime factoring or phase estimation, require extremely
long circuit depth for any practical application, necessitating error
correction. Machine learning can be used as a systematic method to
nonalgorithmically program quantum computers. Quantum machine learning enables
us to perform computations without breaking down an algorithm into its gate
building blocks, eliminating that difficult step and potentially reducing
unnecessary complexity. In addition, we have shown that our machine learning
approach is robust to both noise and to decoherence, which is ideal for running
on inherently noisy NISQ devices whi

In [12]:
df

NameError: name 'df' is not defined